# Hosting ONNX models with Amazon Elastic Inference

Amazon Elastic Inference (EI) is a resource you can attach to your Amazon EC2 instances to accelerate your deep learning (DL) inference workloads. EI allows you to add inference acceleration to an Amazon SageMaker hosted endpoint or Jupyter notebook for a fraction of the cost of using a full GPU instance. It reduces the cost of running deep learning inference by up to 75%. 

Amazon EI provides support for Apache MXNet, TensorFlow and ONNX models. The [Open Neural Network Exchange](https://onnx.ai/) (ONNX) is an open standard format for deep learning models that enables interoperability between deep learning frameworks such as Apache MXNet, Caffe2, Microsoft Cognitive Toolkit(CNTK), PyTorch and more. This means that we can use any of these frameworks to train the model, export these pretrained models in ONNX format and then import them in MXNet for inference. For more information please visit: https://docs.aws.amazon.com/sagemaker/latest/dg/ei.html

In this example, we will use the ResNet-152v1 model from [Deep residual learning for image recognition](https://arxiv.org/abs/1512.03385). This model, alongside many others, can be found at the [ONNX Model Zoo](https://github.com/onnx/models).

We will use the SageMaker Python SDK to host this ONNX model in SageMaker, and perform inference requests.

## Setup

First, we'll get the IAM execution role from our notebook environment, so that SageMaker can access resources in your AWS account later in the example.

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

## The hosting script

We'll need to provide a hosting script that can run on the SageMaker platform. This script will be invoked by SageMaker when we perform inference.

The script we're using here implements two functions:

* `model_fn()` - the SageMaker model server uses this function to load the model
* `transform_fn()` - this function is for using the model to take the input and produce the output

In [ ]:
!pygmentize resnet152.py

## Preparing the model

To create a SageMaker Endpoint, we'll first need to prepare the model to be used in SageMaker.

### Downloading the model

For this example, we will use a pre-trained ONNX model from the [ONNX Model Zoo](https://github.com/onnx/models), where you can find a collection of pre-trained models to work with. Here, we will download the [ResNet-152v1 model](https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet152v1/resnet152v1.onnx) trained on ImageNet dataset.

In [ ]:
import mxnet as mx
mx.test_utils.download('https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet152v1/resnet152v1.onnx')

### Compressing the model data

Now that we have the model data locally, we will need to compress it and upload the tarball to S3 for the SageMaker Python SDK to create a Model

In [ ]:
import tarfile

from sagemaker.session import Session

with tarfile.open('onnx_model.tar.gz', mode='w:gz') as archive:
    archive.add('resnet152v1.onnx')

model_data = Session().upload_data(path='onnx_model.tar.gz', key_prefix='model')

### Creating a SageMaker Python SDK Model instance

With the model data uploaded to S3, we now have everything we need to instantiate a SageMaker Python SDK Model. We'll provide the constructor the following arguments:

* `model_data`: the S3 location of the model data
* `entry_point`: the script for model hosting that we looked at above
* `role`: the IAM role used
* `framework_version`: the MXNet version in use, in this case '1.4.1'

You can read more about creating an `MXNetModel` object in the [SageMaker Python SDK API docs](https://sagemaker.readthedocs.io/en/latest/sagemaker.mxnet.html#mxnet-model).

In [ ]:
from sagemaker.mxnet import MXNetModel

mxnet_model = MXNetModel(model_data=model_data,
                         entry_point='resnet152.py',
                         role=role,
                         py_version='py3',
                         framework_version='1.4.1')

## Creating an inference endpoint and attaching an Elastic Inference(EI) accelerator

Now we can use our `MXNetModel` object to build and deploy an `MXNetPredictor`. This creates a SageMaker Model and Endpoint, the latter of which we can use for performing inference. 

We pass the following arguments to the deploy method:

* `instance_count` - how many instances to back the endpoint.
* `instance_type` - which EC2 instance type to use for the endpoint. For information on supported instance, please check here.
* `accelerator_type` - determines which EI accelerator type to attach to each of our instances. The supported types of accelerators can be found here: https://aws.amazon.com/sagemaker/pricing/instance-types/

### How our models are loaded
By default, the predefined SageMaker MXNet containers have a default `model_fn`, which determines how your model is loaded. The default `model_fn` loads an MXNet Module object with a context based on the instance type of the endpoint.

This applies for EI as well. If an EI accelerator is attached to your endpoint and a custom `model_fn` isn't provided, then the default `model_fn` will load the MXNet Module object with an EI context, `mx.eia()`. This default `model_fn` works with the default save function. If a custom save function was defined, then you may need to write a custom model_fn function. For more information on the `model_fn`, see [SageMaker documentation](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/mxnet/README.rst#model-loading).

### Choosing instance types
We will deploy this model with instance type `ml.m5.xlarge` and `ml.eia1.medium`, and you can experiment with other instance types and EI sizes based on your model requirements. For this model, I found that it requires more CPU memory and so, I chose M5 instance with more memory compared to C5 instances as it is more cost effective. 

In [ ]:
%%time

predictor = mxnet_model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge', accelerator_type='ml.eia1.medium')

## Performing inference

With our Endpoint deployed, we can now send inference requests to it. We'll use one image as an example here.

### Preparing the image

First, we'll download the image (and view it).

In [ ]:
import matplotlib.pyplot as plt
# Download image for inference
img_path = mx.test_utils.download('https://s3.amazonaws.com/onnx-mxnet/examples/mallard_duck.jpg')
img = mx.image.imread(img_path)
plt.imshow(img.asnumpy())

Now, we will preprocess inference image. We will resize it to 256x256, take center crop of 224x224, normalize image, add a dimension to batchify the image.

In [ ]:
from mxnet.gluon.data.vision import transforms
def preprocess(img):
    transform_fn = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img = transform_fn(img)
    img = img.expand_dims(axis=0)
    return img
input_image = preprocess(img)

### Sending the inference request

Now we can use the predictor object to classify the input image:

In [ ]:
scores = predictor.predict(input_image.asnumpy())

To see the inference result, let's download and load `synset.txt` file containing class labels for ImageNet. The top 5 classes generated in order, along with the probabilities are:


In [ ]:
import numpy as np
mx.test_utils.download('https://s3.amazonaws.com/onnx-model-zoo/synset.txt')
with open('synset.txt', 'r') as f:
    labels = [l.rstrip() for l in f]

a = np.argsort(scores)[::-1]

for i in a[0:5]:
        print('class=%s ; probability=%f' %(labels[i],scores[i]))

## Deleting the Endpoint

Since we've reached the end, we'll delete the SageMaker Endpoint to release the instance associated with it.

In [ ]:
predictor.delete_endpoint()